# Import libraries

In [1]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade

import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 947.9/947.9 kB 23.4 MB/s eta 0:00:0000:01


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras
import keras_nlp

from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-08-26 20:46:45.174647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756241205.430064      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756241205.502576      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/2/config.json
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/2/tokenizer.json
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/2/metadata.json
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/2/model.weights.h5
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/2/assets/tokenizer/vocabulary.txt
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/3/config.json
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/3/tokenizer.json
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/3/metadata.json
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/3/model.weights.h5
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/3/assets/tokenizer/vocabulary.txt


# Load the dataset

In [3]:
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

print("Training set shape = {}".format(train_df.shape))
print("Training set memory usage = {:.2f} MB".format(train_df.memory_usage().sum()/1024**2))
print("Testing set shape = {}".format(test_df.shape))
print("Testing set memory usage = {:.2f} MB".format(test_df.memory_usage().sum()/1024**2))

Training set shape = (7613, 5)
Training set memory usage = 0.29 MB
Testing set shape = (3263, 4)
Testing set memory usage = 0.10 MB


### Let's take a look at the first couple of entries.

In [4]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


# Explore the dataset

In [6]:
# an example of what is not a diaster tweet
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [7]:
# an example of a disaster tweet
train_df[train_df["target"]==1]['text'].values[1]

'Forest fire near La Ronge Sask. Canada'

# Preprocess the dataset

In [8]:
BATCH_SIZE = 64
num_training_examples = train_df.shape[0]
train_split = 0.8
val_split = 0.2
steps_per_epoch = int(num_training_examples)*train_split // BATCH_SIZE

EPOCHS = 10
auto = tf.data.experimental.AUTOTUNE

In [9]:
# define the training dataset and validation dataset for both input and prediction values
# define a df for the testing dataset

from sklearn.model_selection import train_test_split

X = train_df["text"]
y = train_df["target"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=val_split, random_state=42)

X_test = test_df["text"]

# Use DistilBERT model with Keras

The text inputs need to be transformed to numerical IDs and arranged in a tensor before being fed into BERT.
BertClassifier already has a preprocessing layer, where it will automatically apply preprocessing to raw inputs during fit(), predict(), and evaluate(). The benefit of DistilledBERT is that it will only use half of the parameters while retaining information from 97% of the performance. While reducing the number of parameters by 40%, it runs 60% faster and preserves 95% of the performance. 

In [10]:
# load the BERT model
preset = "distil_bert_base_en_uncased"

# use shorter sequence length
preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(preset,
                                                                     sequence_length=160,
                                                                     name = "preprocessor_4_tweets")
# Pretrained classifier
classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset,
                                                                  preprocessor=preprocessor,
                                                                  num_classes=2)
classifier.summary()

2025-08-26 20:47:05.739013: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Preprocessor: "preprocessor_4_tweets"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ distil_bert_tokenizer (DistilBertTokenizer)                   │                       Vocab size: 30,522 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "distil_bert_text_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ distil_bert_backbone          │ (None, None, 768)         │      66,362,880 │ padding_mask[0][0],        │
│ (DistilBertBackbone)          │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item (GetItem)            │ (None, 768)               │               0 │ distil_bert_backbone[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pooled_dense (Dense)          │ (None, 768)               │         590,592 │ get_item[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_dropout (Dropout)      │ (None, 768)               │               0 │ pooled_dense[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ logits (Dense)                │ (None, 2)                 │           1,538 │ output_dropout[0][0]       │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 66,955,010 (255.41 MB)

 Trainable params: 66,955,010 (255.41 MB)

 Non-trainable params: 0 (0.00 B)

# Train, predict and fine-tune the model

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# compile
classifier.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(1e-5),
    # optimizer='adam',
    metrics=['accuracy']
)
# fit
history = classifier.fit(
    x=X_train, y=y_train, 
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_data=(X_valid, y_valid)
)

Epoch 1/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 4020s 42s/step - accuracy: 0.6847 - loss: 0.6068 - val_accuracy: 0.8365 - val_loss: 0.4003
Epoch 2/10
13/96 ━━━━━━━━━━━━━━━━━━━━ 53:53 39s/step - accuracy: 0.8217 - loss: 0.3978

# Generate submission file for competition